In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle
import os, sys

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, parent_dir)
os.chdir("..")

2.2.3
3.4.2


In [3]:
os.getcwd()

'/Users/benschaefer/Desktop/epa141-model/final assignment'

In [4]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator, Scenario
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

RES_PATH = "./saved_runs/directed_search/search_worst_case_deventer.pkl"
nfe = 2000
formulation = 3

In [ ]:
with open("./saved_runs/reference_cases/worst_case_deventer.pkl", "rb") as file:
    reference_vals = pickle.load(file)

reference_dict = reference_vals.to_dict()
reference_dict.pop("scenario")
reference_dict.pop("model")
scenario1 = Scenario("Worst case", **reference_dict)
print(reference_dict)
# scenario2 = Scenario("Stable dikes", )


{'A.0_ID flood wave shape': 96, 'A.1_Bmax': 303.445774666745, 'A.1_Brate': 10.0, 'A.1_pfail': 0.9522325105225463, 'A.2_Bmax': 304.52302589651663, 'A.2_Brate': 10.0, 'A.2_pfail': 0.7812595705265573, 'A.3_Bmax': 145.56725538351276, 'A.3_Brate': 1.5, 'A.3_pfail': 0.5853260299693734, 'A.4_Bmax': 273.0092722501388, 'A.4_Brate': 1.0, 'A.4_pfail': 0.6378388121437764, 'A.5_Bmax': 256.7938467227324, 'A.5_Brate': 1.0, 'A.5_pfail': 0.018696290855513813, 'discount rate 0': 4.5, 'EWS_DaysToThreat': 0, 'policy': 'Do Nothing', 'A.1_Expected Annual Damage': 0.0, 'A.1_Dike Investment Costs': 0.0, 'A.1_Expected Number of Deaths': 0.0, 'A.2_Expected Annual Damage': 6792973.611104393, 'A.2_Dike Investment Costs': 0.0, 'A.2_Expected Number of Deaths': 0.009490884604053045, 'A.3_Expected Annual Damage': 39537159.00660494, 'A.3_Dike Investment Costs': 0.0, 'A.3_Expected Number of Deaths': 0.10129154719262956, 'A.4_Expected Annual Damage': 1787621.4969519933, 'A.4_Dike Investment Costs': 0.0, 'A.4_Expected Nu

In [6]:
ema_logging.log_to_stderr(ema_logging.INFO)
os.chdir("/Users/benschaefer/Desktop/epa141-model/final assignment")
# print(os.getcwd())
# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(7)
for lever in dike_model.levers:
    pass
    # print(lever.lower_bound, lever.upper_bound)

[IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False), IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False), IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False), IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable_name=['3_RfR 0'], pff=False), IntegerParameter('4_RfR 0', 0, 1, resolution=None, default=None, variable_name=['4_RfR 0'], pff=False), IntegerParameter('EWS_DaysToThreat', 0, 1, resolution=None, default=None, variable_name=['EWS_DaysToThreat'], pff=False), IntegerParameter('A.1_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.1_DikeIncrease 0'], pff=False), IntegerParameter('A.2_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.2_DikeIncrease 0'], pff=False), IntegerParameter('A.3_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.3_DikeIn

In [7]:
with MultiprocessingEvaluator(dike_model) as evaluator:
            results = evaluator.optimize(
                nfe=nfe,
                searchover='levers',
                reference=scenario1,
                epsilons=[0.1] * len(dike_model.outcomes),
                # convergence=convergence_metrics,
                # constraints=constraints
            )

[MainProcess/INFO] pool started with 8 workers
100%|██████████████████████████████████████| 2000/2000 [03:51<00:00,  8.63it/s]
[MainProcess/INFO] optimization completed, found 73 solutions
[MainProcess/INFO] terminating pool


In [8]:
results

,0_RfR 0,1_RfR 0,2_RfR 0,3_RfR 0,4_RfR 0,EWS_DaysToThreat,A.1_DikeIncrease 0,A.2_DikeIncrease 0,A.3_DikeIncrease 0,A.4_DikeIncrease 0,A.5_DikeIncrease 0,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Expected Number of Deaths
0,1,1,0,1,1,0,0,0,0,0,4,8.510510e+07,3.337712e+07,679700000.0,0.141568
1,0,0,0,1,0,1,0,0,0,6,0,2.217803e+08,1.258646e+07,121200000.0,0.109065
2,0,1,0,1,0,0,0,2,4,5,10,0.000000e+00,1.384672e+08,339000000.0,0.000000
3,1,0,0,0,0,1,0,0,0,1,6,7.078150e+07,4.659447e+07,84600000.0,0.045256
4,0,0,0,0,0,0,0,7,7,3,10,9.480111e+05,1.741670e+08,0.0,0.001267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,0,0,1,0,0,1,0,0,0,1,3,1.597909e+08,3.667233e+07,30700000.0,0.081413
69,0,0,0,1,0,0,0,0,0,0,8,3.112459e+07,4.846370e+07,121200000.0,0.073176
70,0,0,0,1,0,1,0,0,0,1,7,3.008847e+07,5.051431e+07,121200000.0,0.026071
71,0,0,0,1,0,1,0,0,3,2,6,1.743965e+07,7.180379e+07,121200000.0,0.008280


In [9]:
with open(RES_PATH, "wb") as file:
    pickle.dump(results, file)